# Goodreads dataset

In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix
import spacy
import concurrent.futures

np.random.seed(5)
nlp = spacy.load('en_core_web_sm')

In [15]:
#df = pd.read_json('/home/neopux/UHH/datasets/Video_Games_5_proc.json')
df = pd.read_json(r'D:\Datasets\goodreads_reviews\goodreads_reviews_children.json.gz', lines=True)
df_meta = pd.read_json(r'D:\Datasets\goodreads_reviews\goodreads_books_children.json.gz',  lines = True)

In [17]:
df_meta.head(2)

,asin,authors,average_rating,book_id,country_code,description,edition_information,format,image_url,is_ebook,...,publication_year,publisher,ratings_count,series,similar_books,text_reviews_count,title,title_without_series,url,work_id
0,,"[{'role': '', 'author_id': '3041852'}]",4.13,287141,US,"Relates in vigorous prose the tale of Aeneas, ...",,Paperback,https://s.gr-assets.com/assets/nophoto/book/11...,false,...,2006,Yesterday's Classics,46,[],[],7,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls,https://www.goodreads.com/book/show/287141.The...,278578
1,,"[{'role': '', 'author_id': '19158'}]",4.22,6066812,US,"To Kara's astonishment, she discovers that a p...",,Paperback,https://images.gr-assets.com/books/1316637798m...,false,...,2009,Seven Seas,98,[151854],"[948696, 439885, 274955, 12978730, 372986, 216...",6,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...,https://www.goodreads.com/book/show/6066812-al...,701117


In [18]:
df_meta[df_meta['book_id'] == 5]

,asin,authors,average_rating,book_id,country_code,description,edition_information,format,image_url,is_ebook,...,publication_year,publisher,ratings_count,series,similar_books,text_reviews_count,title,title_without_series,url,work_id
53726,,"[{'role': '', 'author_id': '1077326'}, {'role'...",4.53,5,US,Harry Potter's third year at Hogwarts is full ...,,Mass Market Paperback,https://images.gr-assets.com/books/1499277281m...,false,...,2004,Scholastic Inc.,1876252,[162083],"[13759, 65111, 239917, 153800, 393145, 40159, ...",28561,Harry Potter and the Prisoner of Azkaban (Harr...,Harry Potter and the Prisoner of Azkaban (Harr...,https://www.goodreads.com/book/show/5.Harry_Po...,2402163


In [5]:
df.head(2)

,book_id,date_added,date_updated,n_comments,n_votes,rating,read_at,review_id,review_text,started_at,user_id
0,23310161,Tue Nov 17 11:37:35 -0800 2015,Tue Nov 17 11:38:05 -0800 2015,0,7,4,,f4b4b050f4be00e9283c92a814af2670,Fun sequel to the original.,,8842281e1d1347389f2ab93d60773d4d
1,17290220,Sat Nov 08 08:54:03 -0800 2014,Wed Jan 25 13:56:12 -0800 2017,0,4,5,Tue Jan 24 00:00:00 -0800 2017,22d424a2b0057b18fb6ecf017af7be92,One of my favorite books to read to my 5 year ...,,8842281e1d1347389f2ab93d60773d4d


In [10]:
def process_review(s):
    doc = nlp(s)
    tokens = []

    for tok in doc:
        if not tok.is_stop and not tok.is_punct and not tok.like_url and not tok.like_email:
            tokens.append(tok.lemma_.lower().strip() if tok.lemma_ != '-PRON-' else tok.lower_)
    return ' '.join(tokens)

In [13]:
df['review_text_proc'] = df['review_text'].apply(process_review)

In [14]:
df.to_json('D:/Datasets/goodreads_reviews/processed/' + 'goodreads_reviews_children.json')

In [2]:
df = pd.read_json('D:/Datasets/goodreads_reviews/processed/goodreads_reviews_children.json')

In [14]:
df[df['book_id'] == 5]

,book_id,date_added,date_updated,n_comments,n_votes,rating,read_at,review_id,review_text,review_text_proc,started_at,user_id
100330,5,Sat Oct 11 23:32:44 -0700 2008,Wed Aug 20 09:05:25 -0700 2014,0,0,4,Wed Aug 20 09:05:25 -0700 2014,56f9cdab1b6154ef883e9991646669a6,I think I enjoyed rereading book 3 more than I...,i think i enjoy reread book 3 i enjoy book tim...,,5dae9312c2c606a96efe33e0fd68c4a1
10107,5,Mon Jan 18 14:52:40 -0800 2016,Fri May 12 18:07:56 -0700 2017,0,1,5,Fri Dec 02 13:18:24 -0800 2016,5fef558ba32b30e39424c7b3b068c494,This book was always my favorite because of Si...,this book favorite sirius i be sure be i want ...,Sun Nov 27 00:00:00 -0800 2016,e3de9b8acd46c6bfce558ebb2e9eacd4
101774,5,Sun Aug 17 17:37:26 -0700 2008,Tue Nov 24 14:03:44 -0800 2015,0,0,5,Tue Nov 24 14:03:44 -0800 2015,e399dad8a1f7d38ea75a0e846ab44041,I forgot how much I loved this book. It really...,i forget i love book it push series complicate...,Sat Nov 06 00:00:00 -0700 2010,1b709482195baf4bf17dfc001b04537d
101978,5,Wed Nov 07 17:10:07 -0800 2012,Wed Nov 07 17:10:58 -0800 2012,0,0,5,,9844086fc63357dc6e1c2aa0bdbc046f,It was good. Read it a while ago. I didnt read...,it good read ago i not read book movie come,,ef1206c7b7955f61757e50189f2f2eed
101982,5,Sat Mar 19 19:44:10 -0700 2011,Tue Oct 17 11:57:44 -0700 2017,0,7,5,Tue Oct 17 14:37:37 -0700 2017,efbed8b131c76f34720861f910284c96,One of my favorite autumn re-reads!,one favorite autumn read,Tue Oct 10 00:00:00 -0700 2017,6a6cfef9d6413e82d5a37adb3897bdd1
10258,5,Sat Jun 23 15:06:29 -0700 2012,Thu Jan 12 21:30:27 -0800 2017,0,0,5,Mon Sep 14 00:00:00 -0700 2015,f2e33ce8f2f683fea2a66e1f6d2e6533,my absolute favorite book in the series.,absolute favorite book series,Sat Sep 12 00:00:00 -0700 2015,86adc7bacbb26e533c8f71f596ca5259
102667,5,Mon Nov 12 00:39:59 -0800 2012,Mon Nov 12 00:41:12 -0800 2012,0,0,4,,20670398ce1fc70462fbf3421e58ae7a,I started reading the series when I was 12 YO....,i start read series i 12 yo i love immediately...,,1a139f59a1354acbde4eec8facfffd2d
102669,5,Fri Dec 28 20:56:02 -0800 2012,Fri Dec 28 21:06:21 -0800 2012,0,0,4,Fri Dec 28 00:00:00 -0800 2012,7d7eaff60abaaf603e0c5eb8751d1f03,This book amazes me in how intricate all the d...,this book amaze intricate detail fit amazing h...,,66f6205af8610e3d807847cf01091ea0
102799,5,Tue Aug 19 15:54:40 -0700 2008,Wed Jul 08 11:33:16 -0700 2015,0,0,5,Wed Jul 01 00:00:00 -0700 2015,4e958fe764bc488143a0c95de378c90e,Just finished it with my daughters. I have alw...,just finish daughter i love probably favorite ...,Tue Jun 23 00:00:00 -0700 2015,dff447ef9117c4ea26e859a42be4be59
102879,5,Wed Aug 20 19:52:21 -0700 2008,Mon Mar 03 20:16:21 -0800 2014,0,0,5,,9b63310cd57f68b70c397196d2fa4601,2014 read - Reading this genderbent was absolu...,2014 read read genderbent absolutely brilliant...,,ef064bfb98643154be5128160ac840ae


In [13]:
df.groupby('book_id').review_text_proc.count() 

book_id
5           4696
50           788
61            21
93           335
236            8
241           11
244            2
302            7
314            2
329            2
330           31
333            2
334            5
335            1
378         1115
507            1
639           36
881            7
883            1
889            1
903          152
1163           5
1165           2
1166           2
1167          52
1170           1
1852         801
1853          74
1931           7
1946           9
            ... 
36392701       1
36405746       1
36405748       1
36410032       1
36412369       1
36412886       1
36412964       1
36413222       2
36415326       1
36417368       2
36421962       1
36428313       1
36433576       1
36439778       1
36444005       1
36445074       1
36445594       1
36448737       1
36450438       1
36450461       1
36451084       1
36451351       1
36452491       1
36455104       1
36457855       1
36459407       1
36460178       1
364635

In [1]:
df.groupby('book_id').review_text_proc.agg(' '.join)

NameError: name 'df' is not defined

In [38]:
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield l

def get_dataframe(path):
    i = 0
    df = {}
    for d in parse(path):
        dec = json.loads(d.decode('utf8'))
        if not dec['is_read']:
            continue
        dec_cut = {'book_id': dec['book_id'],  'user_id': dec['user_id'], 'rating': dec['rating']}
        df[i] = dec_cut
        i += 1
        if i % 1000000 == 0:
            print(i)
    return pd.DataFrame.from_dict(df, orient='index')

df_meta = get_dataframe(r'D:\Datasets\goodreads_reviews\goodreads_interactions_children.json.gz')

1000000
2000000
3000000
4000000
5000000
6000000


In [50]:
df_meta.head(2)

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,23310161,4
1,8842281e1d1347389f2ab93d60773d4d,18296097,5


In [51]:
df_meta.shape

(6626989, 3)

In [49]:
print('Unique users:', df_meta['user_id'].unique().shape)
print('Unique books:', df_meta['book_id'].unique().shape)

Unique users: (474296,)
Unique books: (123196,)


In [53]:
# sparsity = 1 - #non_zero(A) / #total(A)
sparsity = 1 - 6626989/ (474296 * 123196)

In [55]:
sparsity

0.9998865850826674

In [56]:
df_meta.to_json('D:/Datasets/goodreads_reviews/processed/' + 'goodreads_interactions_children.json')